In [1]:
# script to collect all of the thermo calculations and put them together into a database

In [2]:
import os
import re
import glob

import rmgpy.data.kinetics

# import importlib
# importlib.reload(rmgpy.data.kinetics)

import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

In [3]:
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
kinetics_libs = glob.glob(os.path.join(DFT_DIR, 'kinetics', 'reaction*', 'arkane', 'RMG_libraries'))

In [4]:
# Load the Arkane thermo
entries = []
for i, lib_path in enumerate(kinetics_libs):
    matches = re.search('reaction_([0-9]{4})', lib_path)
    reaction_index = int(matches[1])
    ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
    ark_kinetics_database.load_libraries(lib_path)
    
    # TODO fix bug related to load_libraries not getting the actual name
    for key in ark_kinetics_database.libraries[''].entries.keys():
        entry = ark_kinetics_database.libraries[''].entries[key]
        entry.index = reaction_index
        entries.append(entry)


In [5]:
# compile it all into a single database and a single library which I'll call harris_butane
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.libraries['kinetics'] = rmgpy.data.kinetics.KineticsLibrary()
ark_kinetics_database.libraries['kinetics'].label = 'harris_butane'
ark_kinetics_database.libraries['kinetics'].name = 'harris_butane'
ark_kinetics_database.libraries['kinetics'].entries = OrderedDict()
for entry in entries:
    ark_kinetics_database.libraries['kinetics'].entries[entry.label] = entry

In [6]:
# save the results
output_path = os.path.join(DFT_DIR, 'kinetics')
ark_kinetics_database.save_libraries(output_path, reindex=False)

In [7]:
print(len(entries), 'saved')

105 saved
